In [1]:
import pandas as pd
import numpy as np
from load import Dataset

def add_nominal_noise(df, percentage):
    noisy_df = df.copy()
    nominal_columns = df.select_dtypes(include=['category', 'object']).columns
    for col in nominal_columns:
        if col == 'class':  # Skip the 'class' column
            continue
        unique_values = df[col].apply(lambda x: tuple(x) if isinstance(x, list) else x).unique()
        indices = noisy_df.sample(frac=percentage / 100).index
        noisy_df.loc[indices, col] = np.random.choice(unique_values, size=len(indices))
    return noisy_df

def add_noise_to_train(train_df, percentage, add_noise=True):
    noisy_train = train_df.copy()
    numeric_columns = train_df.select_dtypes(include=[np.number]).columns
    for col in numeric_columns:
        noise = (train_df[col].max() - train_df[col].min()) * (percentage / 100)
        if add_noise:
            noisy_train[col] += noise
        else:
            noisy_train[col] -= noise
    return noisy_train


In [2]:
def generate_noisy_datasets(train_df, valid_df, num_variants=10):
    noisy_datasets = []
    for i in range(1, 11):
        # Add noise
        train_noisy_add = add_numeric_noise(train_df, i, add_noise=True)
        valid_noisy_add = add_numeric_noise(valid_df, i, add_noise=True)
        noisy_datasets.append((train_noisy_add, valid_noisy_add))
        
        # Subtract noise
        train_noisy_subtract = add_numeric_noise(train_df, i, add_noise=False)
        valid_noisy_subtract = add_numeric_noise(valid_df, i, add_noise=False)
        noisy_datasets.append((train_noisy_subtract, valid_noisy_subtract))
    return noisy_datasets

def add_noise_to_train(train_df, percentage, add_noise=True):
    noisy_train = train_df.copy()
    numeric_columns = train_df.select_dtypes(include=[np.number]).columns
    noise = (train_df[numeric_columns].max() - train_df[numeric_columns].min()) * (percentage / 100)
    if add_noise:
        noisy_train[numeric_columns] += noise
    else:
        noisy_train[numeric_columns] -= noise
    return noisy_train

In [3]:
dataset_name = 'cellcycle'  # Replace with your dataset name
dataset = Dataset(dataset_name, nan_strategy='mean')

# # Load train and valid data
# train_df = dataset.train
# valid_df = dataset.valid
# test_df = dataset.test

# # # Generate and store noisy datasets in a list
# # noisy_datasets = generate_noisy_datasets(train_df, valid_df, num_variants=10)

# # # Display the first few rows of the first noisy train and valid dataset as an example
# # noisy_train_1, noisy_valid_1 = noisy_datasets[1]
# # noisy_train_1.head(), noisy_valid_1.head()

In [4]:
# train_df.head()

In [5]:
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

def scale_data(df, method='standard'):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if method == 'standard':
        scaler = StandardScaler()
    elif method == 'minmax':
        scaler = MinMaxScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    return df

# Outliers handling options

def remove_outliers_zscore(df, threshold=3.0):
    z_scores = np.abs(stats.zscore(df.select_dtypes(include=[np.number])))
    filtered_entries = (z_scores < threshold).all(axis=1)
    return df[filtered_entries]

def remove_outliers_iqr(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    filter = (df >= (Q1 - 1.5 * IQR)) & (df <= (Q3 + 1.5 * IQR))
    return df[filter.all(axis=1)]

# Imputing Outliers
def clip_outliers(df, lower_percentile=0.01, upper_percentile=0.99):
    lower_bound = df.quantile(lower_percentile)
    upper_bound = df.quantile(upper_percentile)
    return df.clip(lower=lower_bound, upper=upper_bound, axis=1)

def replace_outliers_with_mean(df, threshold=3.0):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        mean = df[col].mean()
        std = df[col].std()
        outliers = np.abs((df[col] - mean) / std) > threshold
        df.loc[outliers, col] = mean
    return df

In [6]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from hiclass.MultiLabelLocalClassifierPerNode import MultiLabelLocalClassifierPerNode
from hiclass.metrics import f1
from feature_selection import fill_reshape

def add_noise_to_train(train_df, percentage, add_noise=True):
    noisy_train = train_df.copy()
    numeric_columns = train_df.select_dtypes(include=[np.number]).columns
    noise = (train_df[numeric_columns].max() - train_df[numeric_columns].min()) * (percentage / 100)
    if add_noise:
        noisy_train[numeric_columns] += noise
    else:
        noisy_train[numeric_columns] -= noise
    return noisy_train

class NoisePreprocessor(TransformerMixin, BaseEstimator):
    def __init__(self, scaling_method='standard', outlier_method='zscore', imputation_method='mode', outlier_threshold=3.0):
        self.scaling_method = scaling_method
        self.outlier_method = outlier_method
        self.imputation_method = imputation_method
        self.outlier_threshold = outlier_threshold
        self.scaler_ = None

    def fit(self, X, y=None):
        if self.outlier_method in ['zscore', 'iqr', 'clip', 'mean']:
            X_clean, _ = self._remove_outliers(X, y)
        else:
            X_clean = X
        
        return self

    def transform(self, X, y=None):
        X = X.copy()

        if self.outlier_method in ['zscore', 'iqr', 'clip', 'mean']:
            X, y = self._remove_outliers(X, y)
        
        if self.imputation_method in ['mode', 'mean']:
            X = self._impute_nominal_values(X)
        
        if y is not None:
            return X, y
        return X

    def _remove_outliers(self, df, y=None):
        if self.outlier_method == 'zscore':
            return self._remove_outliers_zscore(df, y)
        elif self.outlier_method == 'iqr':
            return self._remove_outliers_iqr(df, y)
        elif self.outlier_method == 'clip':
            return self._clip_outliers(df, y)
        elif self.outlier_method == 'mean':
            return self._replace_outliers_with_mean(df, y)
        else:
            raise ValueError(f"Unknown outlier method: {self.outlier_method}")

    def _remove_outliers_zscore(self, df, y=None):
        numeric_cols = df.select_dtypes(include=[np.number])
        z_scores = np.abs(stats.zscore(numeric_cols))
        filtered_entries = (z_scores < self.outlier_threshold).all(axis=1)
        if y is not None:
            return df.loc[filtered_entries], y.loc[filtered_entries]
        return df.loc[filtered_entries], y

    def _remove_outliers_iqr(self, df, y=None):
        Q1 = df.quantile(0.25)
        Q3 = df.quantile(0.75)
        IQR = Q3 - Q1
        filter = (df >= (Q1 - 1.5 * IQR)) & (df <= (Q3 + 1.5 * IQR))
        filtered_entries = filter.all(axis=1)
        if y is not None:
            return df.loc[filtered_entries], y.loc[filtered_entries]
        return df.loc[filtered_entries], y

    def _clip_outliers(self, df, y=None, lower_percentile=0.01, upper_percentile=0.99):
        lower_bound = df.quantile(lower_percentile)
        upper_bound = df.quantile(upper_percentile)
        df_clipped = df.clip(lower=lower_bound, upper=upper_bound, axis=1)
        return df_clipped, y

    def _replace_outliers_with_mean(self, df, y=None):
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            mean = df[col].mean()
            std = df[col].std()
            outliers = np.abs((df[col] - mean) / std) > self.outlier_threshold
            df.loc[outliers, col] = mean
        return df, y

    def _scale_data(self, df):
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        df.loc[:, numeric_cols] = self.scaler_.transform(df[numeric_cols])
        return df

    def _impute_nominal_values(self, df):
        nominal_cols = df.select_dtypes(include=['category', 'object']).columns
        for col in nominal_cols:
            if self.imputation_method == 'mode':
                mode_value = df[col].mode()[0]
                df[col].fillna(mode_value, inplace=True)
            elif self.imputation_method == 'mean' and df[col].dtype == np.number:
                mean_value = df[col].mean()
                df[col].fillna(mean_value, inplace=True)
        return df


pipeline = Pipeline([
    ('preprocessor', NoisePreprocessor(
        scaling_method=None, 
        outlier_method='mean', 
        imputation_method=None, 
        outlier_threshold=3.0
    )),
    ('classifier', MultiLabelLocalClassifierPerNode(
        local_classifier=LogisticRegression(
            penalty='l2',
            C=0.01,
            solver='lbfgs',
            max_iter=10000
        )
    ))
])

# Introducing noise to x train
noisy_train_x = add_noise_to_train(dataset.x_train(), percentage=10)

pipeline.fit(noisy_train_x, dataset.y_train())

y_pred_test = pipeline.predict(dataset.x_test())
test_accuracy = f1(fill_reshape(dataset.y_test()), fill_reshape(y_pred_test))

results = {
    'f1': test_accuracy
}

print(results)

{'f1': 0.4455458807769591}
